In [1]:
import pandas as pd
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split

In [2]:
rs = 7810

report_date - дата среза фичей

есть есть еденица мы не знаем какая она 1 и 2 или 3е

In [3]:
raw_data = pd.read_csv('data/train_dataset_Самолет.csv', low_memory=False)

In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14456 entries, 0 to 14455
Columns: 2666 entries, report_date to col2663
dtypes: float64(2312), int64(3), object(351)
memory usage: 294.0+ MB


In [5]:
raw_data

,report_date,client_id,target,col1,col2,col3,col4,col5,col6,col7,...,col2654,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663
0,2022-11-01,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
1,2022-11-01,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7616803.0,7616803.0,7616803.0,NaN,NaN,NaN,7616803.0,7616803.0,7616803.0,0.256261
2,2022-05-01,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
3,2022-09-01,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.258682
4,2022-08-01,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,2022-07-01,1241,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.276902
14452,2022-09-01,1969,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260516
14453,2022-02-01,7116,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260005
14454,2021-08-01,7117,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261


### text fields
colomns #

URL: 49-56 221-244 457-464 545-560 1045-1076

От куда перешли: 555 - 576

Поисковые запросы: 577 - 584

Ипотека: 129-152

Платформа 521-544

In [6]:
list_of_text_col = ['col49','col50','col51','col52','col53','col54','col55','col56',
                            'col221','col222','col223','col224','col225','col226','col227','col228','col229','col230','col231','col232','col233','col234','col235','col236','col237','col238','col239','col240','col241','col242','col243','col244',
                            'col457','col458','col459','col460','col461','col462','col463','col464',
                            'col545','col546','col547','col548','col549','col550','col551','col552','col553','col554','col555','col556','col557','col558','col559','col560',
                            'col1045','col1046','col1047','col1048','col1049','col1050','col1051','col1052','col1053','col1054','col1055', 'col1056',
                            'col1057','col1058','col1059','col1060','col1061','col1062','col1063','col1064','col1065','col1066','col1067','col1068','col1069','col1070','col1071','col1072','col1073','col1074','col1075','col1076',
                            'col561','col562','col563','col564','col565','col566','col567','col568','col569','col570','col571','col572','col573','col574','col575','col576',
                            'col577','col578','col579','col580','col581','col582','col583','col584',
                            'col129','col130','col131','col132','col133','col134','col135','col136','col137','col138','col139','col140','col141','col142','col143','col144','col145','col146','col147','col148','col149','col150','col151','col152',
                            'col521','col522','col523','col524','col525','col526','col527','col528','col529','col530','col531','col532','col533','col534','col535','col536','col537','col538','col539','col540','col541','col542','col543','col544']
len(list_of_text_col)

160

In [7]:
def extra_features(raw_data):
    EF = raw_data
    #EF = raw_data.copy()
    EF[list_of_text_col] = EF[list_of_text_col].astype(str)
    EF['text_for_search'] = EF[list_of_text_col].agg(', '.join, axis=1)
    import re
    ipoteka = ['Ипоте', 'Ipote','mortgag']
    comercial = ['comercial']
    ads = ['utm_source']
    family = ['family', 'Детска']
    parking = ['parki']
    apple = ['Mac', 'ios', 'iPhone', 'iPad', "Safari"]
    nonapple = ['Android', 'Windows', 'Linux']
    flats = ['flats']
    price_regex_min = 'price_min=\d+'
    price_regex_max = 'price_max=\d+'

    EF['ipoteka'] = 0
    EF['comercial'] = 0
    EF['ads'] = 0
    EF['family'] = 0
    EF['parking'] = 0
    EF['apple'] = 0
    EF['nonapple'] = 0
    EF['flats']=0
    EF['price_min'] = 0
    EF['price_max'] = 0
    EF['price_mid'] = 0

    for index, row in EF.iterrows():
        count = 0

        for w in ipoteka:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'ipoteka'] = count
        count = 0

        for w in flats:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'flats'] = count
        count = 0

        for w in apple:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'apple'] = count
        count = 0

        for w in nonapple:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'nonapple'] = count
        count = 0

        for w in comercial:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'comercial'] = count
        count = 0

        for w in ads:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'ads'] = count
        count = 0

        for w in family:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'family'] = count
        count = 0

        for w in parking:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'parking'] = count
        count = 0
        min_p = None
        max_p = None
        min_p = re.search(price_regex_min, row['text_for_search'])
        max_p = re.search(price_regex_max, row['text_for_search'])
    

        if min_p != None:
            min_p = min_p.group(0)[10:]
            #print(int(min_p))
            EF.at[index, 'price_min'] = int(min_p)
        if max_p != None:
            max_p = max_p.group(0)[10:]
            #print(int(max_p))
            EF.at[index, 'price_max'] = int(max_p)   
        if min_p != None and max_p != None:
            EF.at[index, 'price_mid']=(int(min_p)+int(max_p))/2
    EF = EF.drop(columns=list_of_text_col)
    EF = EF.drop(columns=['text_for_search'])
    return EF

In [8]:
raw_data = extra_features(raw_data)

In [9]:
raw_data

,report_date,client_id,target,col1,col2,col3,col4,col5,col6,col7,...,comercial,ads,family,parking,apple,nonapple,flats,price_min,price_max,price_mid
0,2022-11-01,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
1,2022-11-01,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,3,0,1,0,0,0.0
2,2022-05-01,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
3,2022-09-01,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
4,2022-08-01,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,2022-07-01,1241,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14452,2022-09-01,1969,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14453,2022-02-01,7116,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14454,2021-08-01,7117,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0


In [10]:
raw_data['price_min'].value_counts()

0           14395
10320072        4
4710000         2
6256590         2
6274419         2
10629965        2
1936228         2
3399373         2
9406491         1
10329893        1
9136488         1
12245433        1
6899999         1
7341214         1
9051794         1
6315401         1
7795401         1
5427247         1
7365288         1
6156457         1
6135421         1
5200000         1
11495127        1
8888025         1
13491002        1
9035698         1
6955042         1
10697746        1
4807282         1
6529444         1
12236269        1
5355636         1
6851881         1
5574038         1
2129850         1
9054809         1
4600000         1
4513120         1
4800000         1
6319504         1
6343994         1
1678348         1
6687583         1
4615533         1
5873982         1
500000          1
5522500         1
7538321         1
4658078         1
15279672        1
6800000         1
5505967         1
4462215         1
Name: price_min, dtype: int64

In [11]:
raw_data['price_max'].value_counts()

0           14395
15279672        3
6606755         2
5000000         2
10103295        2
29149683        2
13228941        1
14597083        1
7000000         1
15244161        1
14313327        1
6924792         1
13970754        1
17506750        1
12929397        1
12156227        1
6465814         1
8275963         1
16510493        1
2021367         1
6900000         1
5537084         1
13063516        1
11975346        1
16903503        1
5698669         1
10580786        1
8045493         1
44155038        1
13371800        1
31941726        1
5442903         1
2979003         1
8964445         1
10873459        1
2129850         1
9158914         1
6339618         1
2113933         1
5498936         1
8555412         1
10561642        1
7547306         1
5447229         1
8020734         1
6600285         1
610000          1
10488885        1
9021696         1
13674616        1
5300000         1
9604269         1
8588776         1
9161749         1
17972703        1
4705465   

In [12]:
raw_data['price_mid'].value_counts()

0.0           14395
6440587.0         2
19889824.0        2
12799872.0        2
6751334.0         2
10512171.0        1
11254144.0        1
13421258.0        1
6949999.5         1
11292687.5        1
11682560.5        1
6620096.5         1
11688622.5        1
11466998.5        1
11624734.5        1
11238149.5        1
7216210.0         1
5125092.5         1
6300617.5         1
1978797.5         1
6578295.0         1
5368542.0         1
12279321.5        1
10431685.5        1
15197252.5        1
5204334.5         1
9808242.0         1
7500267.5         1
27426392.0        1
13420193.0        1
15104486.0        1
19653507.0        1
2328675.5         1
7908163.0         1
8223748.5         1
2129850.0         1
9106861.5         1
4800000.0         1
5426369.0         1
2025080.5         1
5149468.0         1
7437458.0         1
8452818.0         1
6451471.0         1
5031381.0         1
7410367.0         1
6237133.5         1
555000.0          1
8005692.5         1
8280008.5         1


In [13]:
raw_data['flats'].value_counts()

0    13501
1      955
Name: flats, dtype: int64

In [14]:
raw_data['apple'].value_counts()

0    14000
3      370
1       48
2       34
4        4
Name: apple, dtype: int64

In [15]:
raw_data['nonapple'].value_counts()

0    13305
1     1100
2       51
Name: nonapple, dtype: int64

In [16]:
raw_data['ipoteka'].value_counts()

0    13854
1      320
2      177
3      105
Name: ipoteka, dtype: int64

In [17]:
raw_data['ads'].value_counts()

0    14219
1      237
Name: ads, dtype: int64

In [19]:
raw_data['family'].value_counts()

0    14273
1      183
Name: family, dtype: int64

In [20]:
raw_data['parking'].value_counts()

0    14250
1      206
Name: parking, dtype: int64

In [22]:
raw_data['target'].value_counts()

0    14020
1      436
Name: target, dtype: int64

In [23]:
num_unq_clients = raw_data['client_id'].unique().shape[0] # Number of unque clients
num_unq_clients

4817

In [28]:
data = raw_data.drop(columns=['report_date','client_id'])
data

,target,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,comercial,ads,family,parking,apple,nonapple,flats,price_min,price_max,price_mid
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,3,0,1,0,0,0.0
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14452,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14453,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14454,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0


-----------

Удаление столбцов с хеш-суммами, uuid

In [29]:
# Regexp для поиска и удаления хеш сумм, так как это уникальное значение
pattern = r'^[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{12}$'

# Функция для фильтрации колонок по регулярному выражению
def filter_columns(column_name):
    return not data[column_name].astype(str).str.match(pattern).any()

raw_data_wout_regex = data[data.columns[data.columns.to_series().apply(filter_columns)]]

In [30]:
datat = raw_data_wout_regex.copy()
datat

,target,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,comercial,ads,family,parking,apple,nonapple,flats,price_min,price_max,price_mid
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,3,0,1,0,0,0.0
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14452,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14453,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14454,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0


--------------

Split on train test raw data

In [31]:
datat.drop(columns=['target'])

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,...,comercial,ads,family,parking,apple,nonapple,flats,price_min,price_max,price_mid
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,3,0,1,0,0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0


In [32]:
X_train, X_test, y_train, y_test = train_test_split(datat.drop(columns=['target']), datat['target'], test_size=0.3, random_state=rs+20)

In [33]:
Train_data = X_train
Train_data.insert(loc=0, column='target', value=y_train)
Test_data = X_test
Test_data.insert(loc=0, column='target', value=y_test)


In [34]:
Train_data.shape

(10119, 2498)

--------------

Удаление строк с целевым признаком 0, у которых большинство (медианное значение) признаков Nan

In [35]:
# Надо посчитать сколько в среднем заполнено значений для цп 0
median_t0 = Train_data[Train_data['target'] == 0].iloc[:,1:].apply(lambda row: row.notna().sum(), axis=1).median()
median_t0

356.0

In [36]:
data = Train_data[~((Train_data['target'] == 0) & (Train_data.iloc[:, 1:].apply(lambda row: row.notna().sum(), axis=1) <= median_t0))]
data

,target,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,comercial,ads,family,parking,apple,nonapple,flats,price_min,price_max,price_mid
5577,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
8077,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,3,0,1,0,0,0.0
13757,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
1109,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
5236,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8528,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,3,0,1,0,0,0.0
6690,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,0,0,0.0
7267,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
12493,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0


------------

In [37]:
target = data['target']
data = data.drop(['target'], axis=1)

-----

Проход окном

In [38]:
# Дропаем колонки в которых только NaN
good_data = data.drop(data.columns[data.isna().all()], axis=1)

In [39]:
# Серднее окном в 2 колонки
mean_gd = (good_data.count() + good_data.count().shift(-1) + good_data.count().shift(1))/3
mean_gd

col1                 NaN
col2            5.333333
col3           10.333333
col4            9.333333
col5            6.333333
                ...     
nonapple     5205.000000
flats        5205.000000
price_min    5205.000000
price_max    5205.000000
price_mid            NaN
Length: 2411, dtype: float64

In [40]:
selcol = good_data.columns[good_data.count()>=mean_gd] ########### > || >= фичи х2 ~1300
selcol = selcol.to_list()
selcol.append(data.columns[-1]) # добавляем последнюю колонку

In [41]:
good_data = good_data[selcol]
good_data

,col4,col8,col12,col16,col20,col24,col28,col32,col36,col40,...,comercial,ads,family,parking,apple,nonapple,flats,price_min,price_max,price_mid
5577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
8077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,3,0,1,0,0,0.0
13757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
1109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
5236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,3,0,1,0,0,0.0
6690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,0,0,0.0
7267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
12493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0


-------

In [42]:
# Колонки с категориальными данными
catCols = [col for col in good_data.columns if good_data[col].dtype=="object"]
catCols[-10:-1]
#print(f'number cat columns = {len(catCols)}')

['col1448',
 'col1450',
 'col1649',
 'col1651',
 'col1652',
 'col1654',
 'col2193',
 'col2195',
 'col2196']

In [43]:
categorical_columns = good_data[catCols]
numeric_columns = good_data.drop(catCols, axis=1)

-------

In [44]:
# Нормализация данных
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
transform_gd = scaler.fit_transform(numeric_columns)

In [46]:
Test_target = Test_data['target']
Test_data2 = Test_data[numeric_columns.columns]
Test_data3 = scaler.transform(Test_data2)

In [47]:
HistGradientBoostingClassifier().get_params().keys()

dict_keys(['categorical_features', 'early_stopping', 'l2_regularization', 'learning_rate', 'loss', 'max_bins', 'max_depth', 'max_iter', 'max_leaf_nodes', 'min_samples_leaf', 'monotonic_cst', 'n_iter_no_change', 'random_state', 'scoring', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [48]:
import pickle

with open ("data/transform_gd.pkl", 'wb') as fff:
    pickle.dump(transform_gd, file=fff)
with open ("data/target.pkl", 'wb') as fff:
    pickle.dump(target, file=fff)

with open ("data/Test_target.pkl", 'wb') as fff:
    pickle.dump(Test_target, file=fff)
with open ("data/Test_data3.pkl", 'wb') as fff:
    pickle.dump(Test_data3, file=fff)  



In [49]:
with open ("data/transform_gd.pkl", 'rb') as fff:
    transform_gd =  pickle.load(file=fff)
with open ("data/target.pkl", 'rb') as fff:
    target = pickle.load(file=fff)

with open ("data/Test_target.pkl", 'rb') as fff:
    Test_target = pickle.load(file=fff)
with open ("data/Test_data3.pkl", 'rb') as fff:
    Test_data3 = pickle.load(file=fff) 

In [50]:
from sklearn.model_selection import GridSearchCV
if False:
#parameters to tune
    parameters = {
 'max_iter': [150, 175, 200, 225, 250, 300, 350, 400],
 'learning_rate': [0.1],
 'min_samples_leaf': [1, 3, 5, 7],
 'l2_regularization': [0.05, 0.03, 0.07],
 'max_leaf_nodes': [7, 10, 13, 15],
 'max_bins': [190, 200, 210],
 'scoring': ['roc_auc'],
 'random_state' : [rs+20],
    }
#instantiate the gridsearch
    hgb_grid = GridSearchCV(HistGradientBoostingClassifier(), parameters, n_jobs=-1, 
    cv=5, scoring='roc_auc',
    verbose=3, refit=True)
#fit on the grid 
    hgb_grid.fit(transform_gd,target)
    import pickle
    with open ("hgb_grid.pkl", 'wb') as fff:
        pickle.dump(hgb_grid, file=fff)
    with open ("data/hgb_grid.pkl", 'rb') as fff:
    hgb_grid = pickle.load(file=fff)
    print(hgb_grid.best_params_)
    print("Test: ",roc_auc_score(Test_target, hgb_grid.best_estimator_.predict(Test_data3)))


In [53]:

gbc= HistGradientBoostingClassifier(loss='binary_crossentropy', 
                                    max_iter=400,
                                    l2_regularization=0.05,
                                    max_leaf_nodes=15,
                                    max_depth = None,
                                    min_samples_leaf = 7, 
                                    max_bins = 200,
                                    learning_rate = 0.1,  
                                    random_state=rs+20)
gbc.fit(transform_gd,target)

print("Train: ", roc_auc_score(target, gbc.predict(transform_gd)))
print("Test: ",roc_auc_score(Test_target, gbc.predict(Test_data3)))


Train:  1.0
Test:  0.7713746320235505


In [58]:
# import numpy as np
# lll = ['ipoteka','comercial','ads','family','parking','apple','nonapple','flats','price_min','price_max','price_mid']
# for l in lll:
#     print(l,np.corrcoef(np.ndarray(transform_gd[l]), np.ndarray(target))[0,1])

NameError: name 'ndarray' is not defined

0.713510142694692
with extra
0.7176027358430793
0.724238089044956

In [54]:
gbc.predict_proba(Test_data3)

array([[9.91725270e-01, 8.27472950e-03],
       [9.99999195e-01, 8.04885152e-07],
       [1.41307504e-03, 9.98586925e-01],
       ...,
       [9.99956364e-01, 4.36364604e-05],
       [9.99971213e-01, 2.87867480e-05],
       [2.47258953e-03, 9.97527410e-01]])

In [55]:
f1_score(Test_target, gbc.predict(Test_data3))

0.18700787401574803

0.17238001958863858
with extra
0.1750245821042281
0.1780821917808219

In [2]:
test_data = pd.read_csv('test.csv')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 3
